In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import re

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# STEP 1: Get team URLs from league page
def get_team_urls(league_url):
    res = requests.get(league_url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")


    team_links = soup.select('td.hauptlink.no-border-links a')
    
    team_urls = []
    for link in team_links:
        href = link.get("href")
        if href and "/startseite/verein/" in href:
            # Ensure full URL and keep ?saison_id in it
            full_url = f"https://www.transfermarkt.us{href}"
            team_urls.append(full_url)
    return list(set(team_urls))  # remove duplicates

In [3]:
league_urls = {
    "USL1": "https://www.transfermarkt.us/usl-league-one/startseite/wettbewerb/USC3"
}

all_data = []

team_urls = get_team_urls(league_urls["USL1"])

In [4]:
team_urls

['https://www.transfermarkt.us/portland-hearts-of-pine/startseite/verein/124312/saison_id/2024',
 'https://www.transfermarkt.us/one-knoxville-sc/startseite/verein/102031/saison_id/2024',
 'https://www.transfermarkt.us/fc-naples/startseite/verein/120328/saison_id/2024',
 'https://www.transfermarkt.us/texoma-fc/startseite/verein/115284/saison_id/2024',
 'https://www.transfermarkt.us/south-georgia-tormenta-fc/startseite/verein/58894/saison_id/2024',
 'https://www.transfermarkt.us/av-alta-fc/startseite/verein/119858/saison_id/2024',
 'https://www.transfermarkt.us/greenville-triumph-sc/startseite/verein/69055/saison_id/2024',
 'https://www.transfermarkt.us/spokane-velocity-fc/startseite/verein/114568/saison_id/2024',
 'https://www.transfermarkt.us/charlotte-independence/startseite/verein/46717/saison_id/2024',
 'https://www.transfermarkt.us/union-omaha/startseite/verein/77059/saison_id/2024',
 'https://www.transfermarkt.us/richmond-kickers/startseite/verein/16931/saison_id/2024',
 'https://

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

def extract_player_market_values(team_url):
    res = requests.get(team_url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    players = []

    # Find the player rows in the squad table
    table = soup.find("table", class_="items")
    if not table:
        return []

    rows = table.find_all("tr", class_=["odd", "even"])

    for row in rows:
        name_tag = row.find("td", class_="hauptlink")
        value_tag = row.find("td", class_="rechts hauptlink")

        if not name_tag or not value_tag:
            continue

        player_name = name_tag.get_text(strip=True)
        market_value = value_tag.get_text(strip=True)

        # Optional: get player detail page link
        link_tag = name_tag.find("a")
        player_link = (
            "https://www.transfermarkt.us" + link_tag["href"]
            if link_tag and link_tag.has_attr("href")
            else None
        )

        players.append({
            "player_name": player_name,
            "market_value": market_value,
            "player_link": player_link
        })

    return players

overall_data = []
# Example usage
for team_url in team_urls:
    players = extract_player_market_values(team_url)

    print(f"Team URL: {team_url}")
    print(f"Number of players scraped: {len(players)}")
    if players:
        print(players[0])  # Print one player's data as a sanity check

    overall_data.append({
        "team_url": team_url,
        "players": players
    })
    # Optional: Sleep to avoid overwhelming the server
    time.sleep(1)

Team URL: https://www.transfermarkt.us/portland-hearts-of-pine/startseite/verein/124312/saison_id/2024
Number of players scraped: 23
{'player_name': 'Hunter Morse', 'market_value': '€10k', 'player_link': 'https://www.transfermarkt.us/hunter-morse/profil/spieler/637163'}
Team URL: https://www.transfermarkt.us/one-knoxville-sc/startseite/verein/102031/saison_id/2024
Number of players scraped: 24
{'player_name': 'Sean Lewis', 'market_value': '€100k', 'player_link': 'https://www.transfermarkt.us/sean-lewis/profil/spieler/431278'}
Team URL: https://www.transfermarkt.us/fc-naples/startseite/verein/120328/saison_id/2024
Number of players scraped: 23
{'player_name': 'Lalo Delgado', 'market_value': '€150k', 'player_link': 'https://www.transfermarkt.us/lalo-delgado/profil/spieler/748991'}
Team URL: https://www.transfermarkt.us/texoma-fc/startseite/verein/115284/saison_id/2024
Number of players scraped: 26
{'player_name': 'Mason McCready', 'market_value': '€75k', 'player_link': 'https://www.trans

In [6]:
flattened_data = []

for team in overall_data:
    for player in team["players"]:
        flattened_data.append({
            "team_url": team["team_url"],
            **player
        })

overall_df = pd.DataFrame(flattened_data)
overall_df

,team_url,player_name,market_value,player_link
0,https://www.transfermarkt.us/portland-hearts-o...,Hunter Morse,€10k,https://www.transfermarkt.us/hunter-morse/prof...
1,https://www.transfermarkt.us/portland-hearts-o...,Kash Oladapo,-,https://www.transfermarkt.us/kash-oladapo/prof...
2,https://www.transfermarkt.us/portland-hearts-o...,Kemali Green,€75k,https://www.transfermarkt.us/kemali-green/prof...
3,https://www.transfermarkt.us/portland-hearts-o...,Sean Vinberg,€75k,https://www.transfermarkt.us/sean-vinberg/prof...
4,https://www.transfermarkt.us/portland-hearts-o...,Shandon Wright,€50k,https://www.transfermarkt.us/shandon-wright/pr...
...,...,...,...,...
307,https://www.transfermarkt.us/forward-madison-f...,Derek Gebhard,€200k,https://www.transfermarkt.us/derek-gebhard/pro...
308,https://www.transfermarkt.us/forward-madison-f...,Garrett McLaughlin,€175k,https://www.transfermarkt.us/garrett-mclaughli...
309,https://www.transfermarkt.us/forward-madison-f...,Juan Galindrez,€150k,https://www.transfermarkt.us/juan-galindrez/pr...
310,https://www.transfermarkt.us/forward-madison-f...,Adrien Graffin,€25k,https://www.transfermarkt.us/adrien-graffin/pr...


In [7]:
overall_df.to_csv("usl1_market_values_1.csv", index=False)

In [8]:
not_scraped = [
 'https://www.transfermarkt.us/westchester-sc/startseite/verein/122406/saison_id/2024']

In [9]:
overall_data = []
# Example usage
for team_url in not_scraped:
    players = extract_player_market_values(team_url)

    print(f"Team URL: {team_url}")
    print(f"Number of players scraped: {len(players)}")
    if players:
        print(players[0])  # Print one player's data as a sanity check

    overall_data.append({
        "team_url": team_url,
        "players": players
    })
    # Optional: Sleep to avoid overwhelming the server
    time.sleep(1)

Team URL: https://www.transfermarkt.us/westchester-sc/startseite/verein/122406/saison_id/2024
Number of players scraped: 24
{'player_name': 'Dane Jacomen', 'market_value': '€100k', 'player_link': 'https://www.transfermarkt.us/dane-jacomen/profil/spieler/993314'}


In [10]:
flattened_data = []

for team in overall_data:
    for player in team["players"]:
        flattened_data.append({
            "team_url": team["team_url"],
            **player
        })

overall_df = pd.DataFrame(flattened_data)
print(overall_df)
overall_df.to_csv("usl1_market_values_2.csv", index=False)

                                             team_url           player_name  \
0   https://www.transfermarkt.us/westchester-sc/st...          Dane Jacomen   
1   https://www.transfermarkt.us/westchester-sc/st...     Andrew Hammersley   
2   https://www.transfermarkt.us/westchester-sc/st...         Rashid Tetteh   
3   https://www.transfermarkt.us/westchester-sc/st...          Tobi Adewole   
4   https://www.transfermarkt.us/westchester-sc/st...  Andrew Jean-Baptiste   
5   https://www.transfermarkt.us/westchester-sc/st...          Bobby Pierre   
6   https://www.transfermarkt.us/westchester-sc/st...       Sebastián Palma   
7   https://www.transfermarkt.us/westchester-sc/st...           Noah Powder   
8   https://www.transfermarkt.us/westchester-sc/st...            Josh Drack   
9   https://www.transfermarkt.us/westchester-sc/st...        Kemar Lawrence   
10  https://www.transfermarkt.us/westchester-sc/st...         Stephen Payne   
11  https://www.transfermarkt.us/westchester-sc/st..